### Upotrebom više procesa kroz *process pool*, generisanjem taskova na nivou skupa ćelija

Matricu stanja podeliti na N delova (gde je N konfigurabilni parametar) i za svaki deo generisati *task* (poziv funkcije čijim parametrima se definiše koji deo matrice treba obraditi). Funkcija treba da vrati niz koordinata ćelija i njihove nove vrednosti, a matrica za sledeću iteraciju se može kreirati u glavnom programu. Trenutne vrednosti ćelija i suseda se mogu čitati iz deljene matrice.

In [1]:
# Imports:
import random, multiprocessing, time
import numpy as np

from math import sqrt
from functools import partial

In [ ]:
# Global variables:

# Safe to change values:
n_epoch = 50                        # Number of epochs ( iterations ) Conway's game of life have.
n = 30                              # Number of rows and columns ( matrix type: NxN ).

n_splits = 4                        # Number of submatrixes we want to spliting the original matrix - must be power of 2.

# Not safe to change values:
n_neighbours = 8                    # Number of cell neighbours ( TOP_LEFT, TOP, TOP_RIGHT, LEFT, RIGHT, BOT_LEFT, BOT, BOT_RIGHT ).

manager = multiprocessing.Manager()
steps = manager.list()

box_len = int( n // sqrt( n_splits ) )
box_coordinates = [ ( i, j, i + box_len, j + box_len ) for j in range( 0, n , box_len ) for i in range( 0, n, box_len ) ]

print( box_coordinates )

In [4]:
def simulate( box_coordinates, epoch ):
    
    global steps
    
    x1, y1, x2, y2 = box_coordinates
    
    cell_info = list()

    for x in range( x1, x2 ):
        for y in range( y1, y2 ):

            alive = 0
            
            alive += steps[epoch][ ( x - 1 ) % n ][ ( y - 1 ) % n ]         # TOP LEFT
            alive += steps[epoch][ ( x - 1 ) % n ][ y ]                     # TOP
            alive += steps[epoch][ ( x - 1 ) % n ][ ( y + 1 ) % n ]         # TOP RIGHT
            alive += steps[epoch][ x ][ ( y - 1 ) % n ]                     # LEFT      
            alive += steps[epoch][ x ][ ( y + 1 ) % n ]                     # RIGHT
            alive += steps[epoch][ ( x + 1 ) % n ][ ( y - 1 ) % n ]         # BOT LEFT
            alive += steps[epoch][ ( x + 1 ) % n ][ y ]                     # BOT
            alive += steps[epoch][ ( x + 1 ) % n ][ ( y + 1 ) % n ]         # BOT_RIGHT

            curr_cell_state = steps[epoch][x][y]

            # Cell is alive.
            if curr_cell_state == 1:
                if alive < 2 or alive > 3:
                    curr_cell_state = 0
    
            # Cell is dead.
            else:
                if alive == 3:
                    curr_cell_state = 1

            cell_info.append( ( x, y, curr_cell_state ) )

    return cell_info

In [ ]:
# Main

def main():

    global steps

    # Generating initial matrix with random state.
    initial_state = np.array( [ [ random.randint( 0, 1 ) for j in range( n ) ] for i in range( n ) ] )
    steps.append( initial_state )

    pool = multiprocessing.Pool( multiprocessing.cpu_count() )
    
    for epoch in range( n_epoch ):

        partial_simulate = partial( simulate, epoch = epoch )
        
        cell_info = pool.map( partial_simulate, box_coordinates )

        new_step = np.ndarray( shape = ( n, n ), dtype = int )
        
        for cells in cell_info:
            for cell in cells:
                ( x, y, state ) = cell
                new_step[x][y] = state
          

        steps.append( new_step )

        # Progress bar.
        bar = '#' * epoch + ' ' * ( n_epoch - epoch - 1 )
        print( f'\rProgress: [ { bar } ]', end = '' )


    pool.terminate()

if __name__ == "__main__":
    main()

In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
from IPython.display import HTML

def animate( steps ):
  '''
  Animates the array of matrices - each matrix is a single state in simulation.
  '''
  
  def init():
    im.set_data( steps[0] )
    return [ im ]
  
  
  def animate( i ):
    im.set_data( steps[i] )
    return [ im ]


  im = plt.matshow( steps[0], interpolation = 'None', animated = True );
  
  anim = FuncAnimation( im.get_figure(), animate, init_func = init,
                  frames = len( steps ), interval = 1000, blit = True, repeat = False );
                  
  return anim

anim = animate( steps );
HTML( anim.to_html5_video() )